In [3]:
from pathlib import Path
import yaml 
import os
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
import re

In [4]:
PROJ_DIR = Path(os.getcwd()).resolve().parents[0]
PROJ_DIR

PosixPath('/home/anhndt/se334-ai')

In [5]:
def load_dataframe(filename: str, type: str = "raw"):
    file_path = PROJ_DIR / "data" / type / filename
    df = pd.read_excel(file_path)
    return df

In [6]:
train_path = "UIT-VSMEC/train_nor_811.xlsx"
val_path = "UIT-VSMEC/valid_nor_811.xlsx"
test_path = "UIT-VSMEC/test_nor_811.xlsx"

In [7]:
df_train = load_dataframe(train_path)
df_val = load_dataframe(val_path)
df_test = load_dataframe(test_path)
df_train.head(5)

,Unnamed: 0,Emotion,Sentence
0,188,Other,cho mình xin bài nhạc tên là gì với ạ
1,166,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,1345,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,316,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,1225,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...


In [8]:
df_train["Emotion"].value_counts()

Emotion
Enjoyment    1558
Disgust      1071
Other        1021
Sadness       947
Anger         391
Fear          318
Surprise      242
Name: count, dtype: int64

In [9]:
emotion_to_sentiment = {
    "Enjoyment": "positive",
    "Disgust": "negative",
    "Sadness": "negative",
    "Anger": "negative",
    "Fear": "negative",
    "Other": "neutral",
    "Surprise": "neutral",
}
df_train['label'] = df_train["Emotion"].map(emotion_to_sentiment)
df_train['label'].value_counts()

label
negative    2727
positive    1558
neutral     1263
Name: count, dtype: int64

In [10]:
df_train.head()

,Unnamed: 0,Emotion,Sentence,label
0,188,Other,cho mình xin bài nhạc tên là gì với ạ,neutral
1,166,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,negative
2,1345,Disgust,lo học đi . yêu đương lol gì hay lại thích học...,negative
3,316,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :)),positive
4,1225,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...,positive


In [13]:
def load_data(df: pd.DataFrame):
    v_text = []
    v_label = []

    label_map = {"positive": 1, "neutral": 0, "negative": 2}

    for _, row in df.iterrows():
        text = row["Sentence"]
        label = label_map[row["label"]]

        v_text.append(text)
        v_label.append(int(label))

    return v_text, v_label
